In [2]:
import pandas as pd

In [3]:
file_path = []
file_path.append(r'C:\Users\ADMIN\Desktop\TASK_CLEAN_DATA\transactions\transactions_pp02.csv')
file_path.append(r'C:\Users\ADMIN\Desktop\TASK_CLEAN_DATA\transactions\transactions_pp04.csv')
file_path.append(r'C:\Users\ADMIN\Desktop\TASK_CLEAN_DATA\transactions\transactions_pp05.csv')
file_path.append(r'C:\Users\ADMIN\Desktop\TASK_CLEAN_DATA\transactions\transactions_pp06.csv')

In [4]:
# print(file_path[0])
df_pp02 = pd.read_csv(file_path[0])
df_pp02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3293 entries, 0 to 3292
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   transaction_id               3293 non-null   object
 1   transaction_event_code       3293 non-null   object
 2   transaction_initiation_date  3293 non-null   object
 3   transaction_updated_date     3293 non-null   object
 4   transaction_amount           3293 non-null   object
 5   transaction_status           3293 non-null   object
 6   transaction_subject          88 non-null     object
 7   ending_balance               3293 non-null   object
 8   available_balance            3293 non-null   object
 9   protection_eligibility       3293 non-null   int64 
 10  payer_info                   3293 non-null   object
 11  shipping_info                3293 non-null   object
 12  cart_info                    3293 non-null   object
 13  store_info                   3293

In [ ]:
transactions = {}
for col in df_pp02:
    for i in range(19):
        print(df_pp02[col].iloc[i])
        transactions[col] = df_pp02[col].iloc[0]
        break

print(transactions)


78W97559FU6028206
{'transaction_id': '78W97559FU6028206'}


In [26]:
import yaml
import pyodbc
import urllib.parse
from sqlalchemy import create_engine

server = "DESKTOP-2PSQA2P\\MSSQL_SERVER"
database = "TASK"
username = "DESKTOP-2PSQA2P\\ADMIN"

connection_string = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    Trusted_Connection=yes;
"""
conn = pyodbc.connect(connection_string)
print("Connection succesful")

params = urllib.parse.quote_plus(connection_string)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


Connection succesful


In [27]:
table_name = 'transactions_pp02'
df_pp02.to_sql(table_name, con=engine, if_exists='replace', index=False)

103

xử lý cột transaction_amount thành một bảng, một hàng chứa currency_code và value

In [33]:
import json

transaction_amount = []
data = {"currency_code": "USD", "value": "52.24"}

# print(df_pp02['transaction_amount'])

for index, item in enumerate(df_pp02['transaction_amount']):
    parsed_item = json.loads(item)
    # print(parsed_item)
    transaction_amount.append({
        'currency_code' : parsed_item['currency_code'],
        'value' : parsed_item['value']
    })

# print(transaction_amount)
df_tmp = pd.json_normalize(transaction_amount)
df_tmp.tail()



,currency_code,value
3288,USD,52.24
3289,USD,52.24
3290,USD,52.24
3291,USD,52.24
3292,USD,52.24


xử lý cột ending_balance thành một bảng, một hàng chứa currency_code và value

In [34]:
import json

ending_balance = []
data = {"currency_code": "USD", "value": "52.24"}

# print(df_pp02['transaction_amount'])

for index, item in enumerate(df_pp02['ending_balance']):
    parsed_item = json.loads(item)
    # print(parsed_item)
    ending_balance.append({
        'currency_code' : parsed_item['currency_code'],
        'value' : parsed_item['value']
    })

# print(transaction_amount)
df_tmp = pd.json_normalize(ending_balance)
df_tmp.tail()

,currency_code,value
3288,USD,2153.89
3289,USD,2153.89
3290,USD,2153.89
3291,USD,2153.89
3292,USD,2153.89


xử lý cột available_balance thành một bảng, một hàng chứa currency_code và value

In [36]:
import json

available_balance = []
data = {"currency_code": "USD", "value": "52.24"}

# print(df_pp02['transaction_amount'])

for index, item in enumerate(df_pp02['available_balance']):
    parsed_item = json.loads(item)
    # print(parsed_item)
    available_balance.append({
        'currency_code' : parsed_item['currency_code'],
        'value' : parsed_item['value']
    })

# print(transaction_amount)
df_tmp = pd.json_normalize(available_balance)
df_tmp.head()

,currency_code,value
0,USD,653.89
1,USD,653.89
2,USD,653.89
3,USD,653.89
4,USD,653.89


xử lý cột payer_info, chia mỗi thuộc tính thành mỗi cột

In [47]:
import json

payer_info = []
data = {"account_id": "899UJ86SL7G38", "email_address": "lathanhittmoore@gmail.com", "address_status": "N", 
        "payer_status": "N", "payer_name": {"given_name": "Lathan", "surname": "Moore", "alternate_full_name": "Lathan Moore"}, 
        "country_code": "US"}
data2 = {"address_status": "N", "payer_name": {}, 
         "address": {"line1": "2676 elston street", "city": "Livermore", "state": "CA", "country_code": "US", "postal_code": "94550"}}



# print(df_pp02['transaction_amount'])

for index, item in enumerate(df_pp02['payer_info']):
    parsed_item = json.loads(item)
    # print(parsed_item)
    account_id = email_address = address_status = payer_status = payer_name = given_name = 'NULL'
    surname = alternate_full_name = country_code = line1 = city = state = postal_code = 'NULL'
    if 'account_id' in parsed_item:
        account_id = parsed_item['account_id']
    if 'email_address' in parsed_item:
        email_address = parsed_item['email_address']
    if 'address_status' in parsed_item:
        address_status = parsed_item['address_status']
    if 'payer_status' in parsed_item:
        payer_status = parsed_item['payer_status']
    if 'payer_name' in parsed_item:
        # payer_name = parsed_item['payer_name']
        for item2 in parsed_item['payer_name']:
            if 'given_name' in parsed_item['payer_name']:
                given_name = parsed_item['payer_name']['given_name']
            if 'surname' in parsed_item['payer_name']:
                surname = parsed_item['payer_name']['surname']
            if 'alternate_full_name' in parsed_item['payer_name']:
                alternate_full_name = parsed_item['payer_name']['alternate_full_name']
    if 'country_code' in parsed_item:
        country_code = parsed_item['country_code']
    if 'address' in parsed_item:
        if 'line1' in parsed_item['address']:
            line1 = parsed_item['address']['line1']
        if 'city' in parsed_item['address']:
            city = parsed_item['address']['city']
        if 'state' in parsed_item['address']:
            state = parsed_item['address']['state']
        if 'country_code' in parsed_item['address']:
            country_code = parsed_item['address']['country_code']
        if 'postal_code' in parsed_item['address']:
            postal_code = parsed_item['address']['postal_code']

    payer_info.append({
        'account_id': account_id,
        'email_address': email_address,
        'address_status': address_status,
        'payer_status': payer_status,
        'payer_name': payer_name,
        'given_name': given_name,
        'surname': surname,
        'alternate_full_name': alternate_full_name,
        'country_code': country_code,
        'line1': line1,
        'city': city,
        'state': state,
        'postal_code': postal_code
    })
    

# print(payer_info)
df_tmp = pd.json_normalize(payer_info)
df_tmp

,account_id,email_address,address_status,payer_status,payer_name,given_name,surname,alternate_full_name,country_code,line1,city,state,postal_code
0,899UJ86SL7G38,lathanhittmoore@gmail.com,N,N,NULL,Lathan,Moore,Lathan Moore,US,NULL,NULL,NULL,NULL
1,899UJ86SL7G38,lathanhittmoore@gmail.com,N,N,NULL,Lathan,Moore,Lathan Moore,US,NULL,NULL,NULL,NULL
2,899UJ86SL7G38,lathanhittmoore@gmail.com,N,N,NULL,Lathan,Moore,Lathan Moore,US,NULL,NULL,NULL,NULL
3,899UJ86SL7G38,lathanhittmoore@gmail.com,N,N,NULL,Lathan,Moore,Lathan Moore,US,NULL,NULL,NULL,NULL
4,899UJ86SL7G38,lathanhittmoore@gmail.com,N,N,NULL,Lathan,Moore,Lathan Moore,US,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288,899UJ86SL7G38,lathanhittmoore@gmail.com,N,N,NULL,Lathan,Moore,Lathan Moore,US,NULL,NULL,NULL,NULL
3289,899UJ86SL7G38,lathanhittmoore@gmail.com,N,N,NULL,Lathan,Moore,Lathan Moore,US,NULL,NULL,NULL,NULL
3290,899UJ86SL7G38,lathanhittmoore@gmail.com,N,N,NULL,Lathan,Moore,Lathan Moore,US,NULL,NULL,NULL,NULL
3291,899UJ86SL7G38,lathanhittmoore@gmail.com,N,N,NULL,Lathan,Moore,Lathan Moore,US,NULL,NULL,NULL,NULL


In [ ]:

shipping_info = []
data = {"name": "Gordon Sears", "address": {"line1": "439 Koberlin St", "city": "San Angelo", "country_code": "US", "postal_code": "76903"}}

# print(df_pp02['transaction_amount'])

for index, item in enumerate(df_pp02['shipping_info']):
    parsed_item = json.loads(item)
    # print(parsed_item)
    shipping_info.append({
        
    })

# print(transaction_amount)
df_tmp = pd.json_normalize(available_balance)
df_tmp.head()